In [241]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup, Comment
import requests
import time

Below I am reading in CSV files that I downloaded from profootballfocus.com for the years and positions that I am looking at, from 2018-2009.  Each CSV represented one season of play for one position, so you can see below that I am joining all the years together for each position, creating dataframes for each differnet category (passing, rushing, receiving, defense), and creating a unique key for each player and the year they are playing in that I will use later to map this data to my larger dataset. 

In [242]:
draft_years = ['2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006']
passing_df = pd.DataFrame()

for year in draft_years: 
    temp_df = pd.read_csv('../data/targets/passing/passing_summary_' + year + '.csv')
    temp_df['key'] = temp_df['player'] + year
    temp_df['year'] = int(year)
    passing_df = pd.concat([passing_df, temp_df])

In [243]:
draft_years = ['2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006']
rushing_df = pd.DataFrame()

for year in draft_years: 
    temp_df = pd.read_csv('../data/targets/rushing/rushing_summary_' + year + '.csv')
    temp_df['key'] = temp_df['player'] + year
    temp_df['year'] = int(year)
    rushing_df = pd.concat([rushing_df, temp_df])

In [244]:
draft_years = ['2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006']
receiving_df = pd.DataFrame()

for year in draft_years: 
    temp_df = pd.read_csv('../data/targets/receiving/receiving_summary_' + year + '.csv')
    temp_df['key'] = temp_df['player'] + year
    temp_df['year'] = int(year)
    receiving_df = pd.concat([receiving_df, temp_df])

In [245]:
draft_years = ['2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006']
defense_df = pd.DataFrame()

for year in draft_years: 
    temp_df = pd.read_csv('../data/targets/defense/defense_summary_' + year + '.csv')
    temp_df['key'] = temp_df['player'] + year
    temp_df['year'] = int(year)
    defense_df = pd.concat([defense_df, temp_df])

In [246]:
years = ['2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005']
nfl_standings = pd.DataFrame()

for year in years: 
    temp_df = pd.read_csv('../data/college/nfl_standings/nfl_standings_' + year + '.csv')
    temp_df['year'] = year
    nfl_standings = pd.concat([nfl_standings, temp_df], sort=False)
nfl_standings.to_csv('../data/college/nfl_standings/all_nfl_standings.csv')

In [247]:
nfl_standings.head()

Tm   W   L    T   W-L%   PF   PA   PD  MoV  SoS  SRS  \
0  New England Patriots*  11   5  0.0  0.688  436  325  111  6.9 -1.8  5.2   
1         Miami Dolphins   7   9  0.0  0.438  319  433 -114 -7.1 -1.7 -8.8   
2          Buffalo Bills   6  10  0.0  0.375  269  374 -105 -6.6 -0.3 -6.9   
3          New York Jets   4  12  0.0  0.250  333  441 -108 -6.8 -1.1 -7.8   
4      Baltimore Ravens*  10   6  0.0  0.625  389  287  102  6.4  0.6  7.0   

   OSRS  DSRS  year  
0   3.1   2.1  2018  
1  -3.6  -5.2  2018  
2  -6.3  -0.6  2018  
3  -2.0  -5.9  2018  
4   0.6   6.4  2018

In [248]:
nfl_standings = nfl_standings.reset_index()
nfl_standings['key'] = np.nan

teams = {
    'Patriots',
    'Dolphins',
    'Bills',
    'Jets',
    'Ravens',
    'Steelers',
    'Browns',
    'Bengals',
    'Texans', 
    'Colts',
    'Titans', 
    'Jaguars', 
    'Chiefs',
    'Chargers',
    'Broncos',
    'Raiders',
    'Cowboys',
    'Eagles',
    'Redskins',
    'Giants',
    'Bears', 
    'Vikings',
    'Packers',
    'Lions',
    'Saints',
    'Panthers',
    'Falcons',
    'Buccaneers',
    'Rams',
    'Seahawks',
    '49ers',
    'Cardinals'
}
    
for index, row in nfl_standings.iterrows():
    for team in teams:
        if team in row['Tm']:
            nfl_standings.loc[index, 'key'] = team + row['year']

In [249]:
all_drafts = pd.read_csv('../data/college/all_drafts.csv') 
all_drafts.drop(columns='Unnamed: 0', inplace=True)

all_drafts['key'] = np.nan
all_drafts['nfl_team'] = np.nan

abr_keys = {
    'NWE': 'Patriots',
    'MIA': 'Dolphins',
    'BUF': 'Bills',
    'NYJ': 'Jets',
    'BAL': 'Ravens',
    'PIT': 'Steelers',
    'CLE': 'Browns',
    'CIN': 'Bengals',
    'HOU': 'Texans', 
    'IND': 'Colts',
    'TEN': 'Titans', 
    'JAX': 'Jaguars', 
    'KAN': 'Chiefs',
    'LAC': 'Chargers',
    'SDG': 'Chargers',
    'DEN': 'Broncos',
    'OAK': 'Raiders',
    'DAL': 'Cowboys',
    'PHI': 'Eagles',
    'WAS': 'Redskins',
    'NYG': 'Giants',
    'CHI': 'Bears', 
    'MIN': 'Vikings',
    'GNB': 'Packers',
    'DET': 'Lions',
    'NOR': 'Saints',
    'CAR': 'Panthers',
    'ATL': 'Falcons',
    'TAM': 'Buccaneers',
    'LAR': 'Rams',
    'STL': 'Rams',
    'SEA': 'Seahawks',
    'SFO': '49ers',
    'ARI': 'Cardinals'
}

for index, row in all_drafts.iterrows():
    for key in abr_keys:
        if row['team_nfl']==key:
            all_drafts.loc[index, 'key'] = abr_keys[key] + str(row['draft_year'] - 1)
            all_drafts.loc[index, 'nfl_team'] = abr_keys[key]

In [250]:
nfl_standings = nfl_standings[['key', 'W-L%']]

all_drafts = pd.merge(all_drafts, nfl_standings, on='key', how='left')
all_drafts.drop(columns='key', inplace=True)

all_drafts = all_drafts.rename(columns={'W-L%': 'nfl_team_win_perc'})

In [251]:
all_player_combine = pd.read_csv('../data/college/all_player_combine.csv').drop(columns='Unnamed: 0')

In [252]:
all_drafts = pd.concat([all_drafts, all_player_combine.drop(columns='player')], axis=1)

In [253]:
all_drafts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2718 entries, 0 to 2717
Data columns (total 20 columns):
college_stats_url    2339 non-null object
draft_age            2655 non-null float64
draft_overall        2718 non-null int64
draft_round          2718 non-null int64
draft_year           2718 non-null int64
player               2718 non-null object
position             2718 non-null object
pro_stats_url        2661 non-null object
school               2711 non-null object
team_nfl             2718 non-null object
nfl_team             2718 non-null object
nfl_team_win_perc    2718 non-null float64
bench_reps           2113 non-null float64
broad_jump           2141 non-null float64
cone                 2072 non-null float64
forty_yd             2146 non-null float64
height               2193 non-null float64
shuttle              2077 non-null float64
vertical             2142 non-null float64
weight               2193 non-null float64
dtypes: float64(10), int64(3), object(7)
memor

In [254]:
all_drafts.head()

college_stats_url  draft_age  \
0  http://www.sports-reference.com/cfb/players/ma...       21.0   
1  http://www.sports-reference.com/cfb/players/re...       21.0   
2  http://www.sports-reference.com/cfb/players/vi...       23.0   
3  http://www.sports-reference.com/cfb/players/aj...       22.0   
4  http://www.sports-reference.com/cfb/players/ve...       22.0   

   draft_overall  draft_round  draft_year          player position  \
0              1            1        2006  Mario Williams       DE   
1              2            1        2006     Reggie Bush       RB   
2              3            1        2006     Vince Young       QB   
3              5            1        2006       A.J. Hawk       LB   
4              6            1        2006    Vernon Davis       TE   

                                       pro_stats_url              school  \
0  https://www.pro-football-reference.com/players...  North Carolina St.   
1  https://www.pro-football-reference.com/players...                 USC   
2  https://www.pro-football-reference.com/players...               Texas   
3  https://www.pro-football-reference.com/players...            Ohio St.   
4  https://www.pro-football-reference.com/players...            Maryland   

  team_nfl nfl_team  nfl_team_win_perc  bench_reps  broad_jump  cone  \
0      HOU   Texans              0.125        35.0       120.0  7.19   
1      NOR   Saints              0.188         0.0         0.0  0.00   
2      TEN   Titans              0.250         0.0         0.0  0.00   
3      GNB  Packers              0.250        24.0       115.0  6.82   
4      SFO    49ers              0.250        33.0       128.0  7.00   

   forty_yd  height  shuttle  vertical  weight  
0      4.70    79.0     4.36      40.5   295.0  
1      4.37    71.0     0.00       0.0   201.0  
2      4.48    77.0     0.00       0.0   229.0  
3      4.59    73.0     3.96      40.0   248.0  
4      4.38    75.0     4.17      42.0   254.0

Below I drop a subset of data where the player data I scraped off of pro-football-reference did not have associated college stats records.  Since I cannot use these players in my model I dropped those datapoints. 

In [255]:
len(all_drafts)

2718

In [256]:
len(all_drafts) - len(all_drafts.dropna(subset=['college_stats_url']))

379

In [257]:
dropped_college_stats = all_drafts[all_drafts['college_stats_url'].isnull()==True]

In [258]:
all_drafts.dropna(subset=['college_stats_url'], inplace=True)

In [259]:
all_drafts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2339 entries, 0 to 2717
Data columns (total 20 columns):
college_stats_url    2339 non-null object
draft_age            2288 non-null float64
draft_overall        2339 non-null int64
draft_round          2339 non-null int64
draft_year           2339 non-null int64
player               2339 non-null object
position             2339 non-null object
pro_stats_url        2292 non-null object
school               2339 non-null object
team_nfl             2339 non-null object
nfl_team             2339 non-null object
nfl_team_win_perc    2339 non-null float64
bench_reps           1922 non-null float64
broad_jump           1950 non-null float64
cone                 1881 non-null float64
forty_yd             1955 non-null float64
height               2002 non-null float64
shuttle              1886 non-null float64
vertical             1951 non-null float64
weight               2002 non-null float64
dtypes: float64(10), int64(3), object(7)
memor

Below I am looping through the larger all_drafts dataframe and encoding columns for grades, and number of games for each player for their first, second, third, and fourth years in the NFL. 

In [260]:
all_drafts['games_y1'] = np.nan
all_drafts['ov_grade_y1'] = np.nan
all_drafts['games_y2'] = np.nan
all_drafts['ov_grade_y2'] = np.nan
all_drafts['games_y3'] = np.nan
all_drafts['ov_grade_y3'] = np.nan
all_drafts['games_y4'] = np.nan
all_drafts['ov_grade_y4'] = np.nan

for index, row in all_drafts.iterrows():
    print(row['player'])
    
    if row['position'] == 'QB':
        
        try:
            passing_mask = passing_df['key']== row['player']+str(row['draft_year'])
            passing_index = passing_df[passing_mask].index[0]
            all_drafts.loc[index, 'games_y1'] = passing_df[passing_mask].loc[passing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y1'] = passing_df[passing_mask].loc[passing_index, 'grades_offense']
        except IndexError:
            pass
        
        try:
            passing_mask = passing_df['key'] == row['player']+str(row['draft_year']+1)
            passing_index = passing_df[passing_mask].index[0]
            all_drafts.loc[index, 'games_y2'] = passing_df[passing_mask].loc[passing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y2'] = passing_df[passing_mask].loc[passing_index, 'grades_offense']
        except IndexError:
            pass

        try:
            passing_mask = passing_df['key'] == row['player']+str(row['draft_year']+2)
            passing_index = passing_df[passing_mask].index[0]
            all_drafts.loc[index, 'games_y3'] = passing_df[passing_mask].loc[passing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y3'] = passing_df[passing_mask].loc[passing_index, 'grades_offense']
        except IndexError:
            pass
        
        try:
            passing_mask = passing_df['key'] == row['player']+str(row['draft_year']+2)
            passing_index = passing_df[passing_mask].index[0]
            all_drafts.loc[index, 'games_y4'] = passing_df[passing_mask].loc[passing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y4'] = passing_df[passing_mask].loc[passing_index, 'grades_offense']
        except IndexError:
            pass
    
    
    elif (row['position'] == 'RB') | (row['position'] == 'FB'):
        try:
            rushing_mask = rushing_df['key']== row['player']+str(row['draft_year'])
            rushing_index = rushing_df[rushing_mask].index[0]
            all_drafts.loc[index, 'games_y1'] = rushing_df[rushing_mask].loc[rushing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y1'] = rushing_df[rushing_mask].loc[rushing_index, 'grades_offense']
        except:
            pass
        
        try:
            rushing_mask = rushing_df['key']== row['player']+str(row['draft_year']+1)
            rushing_index = rushing_df[rushing_mask].index[0]
            all_drafts.loc[index, 'games_y2'] = rushing_df[rushing_mask].loc[rushing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y2'] = rushing_df[rushing_mask].loc[rushing_index, 'grades_offense']
        except IndexError:
            pass
        
        try:
            rushing_mask = rushing_df['key']== row['player']+str(row['draft_year']+2)
            rushing_index = rushing_df[rushing_mask].index[0]
            all_drafts.loc[index, 'games_y3'] = rushing_df[rushing_mask].loc[rushing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y3'] = rushing_df[rushing_mask].loc[rushing_index, 'grades_offense']
            
        except IndexError:
            pass
            
        try:
            rushing_mask = rushing_df['key']== row['player']+str(row['draft_year']+3)
            rushing_index = rushing_df[rushing_mask].index[0]
            all_drafts.loc[index, 'games_y4'] = rushing_df[rushing_mask].loc[rushing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y4'] = rushing_df[rushing_mask].loc[rushing_index, 'grades_offense']
            
        except:
            pass
        
    
    elif (row['position'] == 'WR') | (row['position'] == 'TE'):
        try:
            receiving_mask = receiving_df['key']== row['player']+str(row['draft_year'])
            receiving_index = receiving_df[receiving_mask].index[0]
            all_drafts.loc[index, 'games_y1'] = receiving_df[receiving_mask].loc[receiving_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y1'] = receiving_df[receiving_mask].loc[receiving_index, 'grades_offense']
        except IndexError:
            pass
        
        try:
            receiving_mask = receiving_df['key']== row['player']+str(row['draft_year']+1)
            receiving_index = receiving_df[receiving_mask].index[0]
            all_drafts.loc[index, 'games_y2'] = receiving_df[receiving_mask].loc[receiving_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y2'] = receiving_df[receiving_mask].loc[receiving_index, 'grades_offense']
        except IndexError:
            pass
        
        try:
            receiving_mask = receiving_df['key']== row['player']+str(row['draft_year']+2)
            receiving_index = receiving_df[receiving_mask].index[0]
            all_drafts.loc[index, 'games_y3'] = receiving_df[receiving_mask].loc[receiving_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y3'] = receiving_df[receiving_mask].loc[receiving_index, 'grades_offense']
        except IndexError:
            pass
            
        try:
            receiving_mask = receiving_df['key']== row['player']+str(row['draft_year']+3)
            receiving_index = receiving_df[receiving_mask].index[0]
            all_drafts.loc[index, 'games_y4'] = receiving_df[receiving_mask].loc[receiving_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y4'] = receiving_df[receiving_mask].loc[receiving_index, 'grades_offense']
            
        except IndexError:
            pass
        
    elif row['position'] in ['CB', 'DE','ILB', 'S', 'DT', 'OLB', 'LB', 'DB', 'DL', 'NT']:
        try:
            defense_mask = defense_df['key']== row['player']+str(row['draft_year'])
            defense_index = defense_df[defense_mask].index[0]
            all_drafts.loc[index, 'games_y1'] = defense_df[defense_mask].loc[defense_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y1'] = defense_df[defense_mask].loc[defense_index, 'grades_defense']
        except IndexError:
            pass
        
        try:
            defense_mask = defense_df['key']== row['player']+str(row['draft_year']+1)
            defense_index = defense_df[defense_mask].index[0]
            all_drafts.loc[index, 'games_y2'] = defense_df[defense_mask].loc[defense_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y2'] = defense_df[defense_mask].loc[defense_index, 'grades_defense']
        except IndexError:
            pass
        
        try:
            defense_mask = defense_df['key']== row['player']+str(row['draft_year']+2)
            defense_index = defense_df[defense_mask].index[0]
            all_drafts.loc[index, 'games_y3'] = defense_df[defense_mask].loc[defense_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y3'] = defense_df[defense_mask].loc[defense_index, 'grades_defense']
        except IndexError:
            pass
            
        try:
            defense_mask = defense_df['key']== row['player']+str(row['draft_year']+3)
            defense_index = defense_df[defense_mask].index[0]
            all_drafts.loc[index, 'games_y4'] = defense_df[defense_mask].loc[defense_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y4'] = defense_df[defense_mask].loc[defense_index, 'grades_defense']            
        except IndexError:
            pass
            

Mario Williams
Reggie Bush
Vince Young
A.J. Hawk
Vernon Davis
Michael Huff
Donte Whitner
Ernie Sims
Matt Leinart
Jay Cutler
Haloti Ngata
Kamerion Wimbley
Brodrick Bunkley
Tye Hill
Jason Allen
Chad Greenway
Bobby Carpenter
Antonio Cromartie
Tamba Hali
Laurence Maroney
Manny Lawson
Santonio Holmes
John McCargo
DeAngelo Williams
Marcedes Lewis
Joseph Addai
Kelly Jennings
Mathias Kiwanuka
DeMeco Ryans
D'Qwell Jackson
Chad Jackson
Thomas Howard
Daniel Bullocks
Roman Harper
Sinorice Moss
LenDale White
Joe Klopfenstein
Cedric Griffin
Kellen Clemens
Greg Jennings
Anthony Fasano
Bernard Pollard
Devin Hester
Richard Marshall
Maurice Jones-Drew
Tony Scheffler
Tim Jennings
Darryl Tapp
Tarvaris Jackson
Abdul Hodge
Claude Wroten
Ashton Youboty
Leonard Pope
Dusty Dvoracek
Brian Calhoun
Anthony Schlegel
Jon Alston
Travis Wilson
Jerious Norwood
Clint Ingram
Charlie Whitehurst
Derek Hagan
Anthony Smith
Brandon Williams
Brodie Croyle
David Thomas
James Anderson
Maurice Stovall
Frostee Rucker
Dominique By

Derrick Williams
Brandon Tate
Mike Wallace
Asher Allen
Patrick Turner
Jared Cook
Chris Owens
Deon Butler
Jerraud Powers
Corvey Irvin
Ryan Mouton
Rashad Johnson
Keenan Lewis
Tyrone McKenzie
Chase Coffman
Juaquin Iglesias
Travis Beckum
Stephen McGee
Donald Washington
Kaluka Maiava
Henry Melton
Mike Thomas
Brian Hartline
Victor Butler
Mike Goodson
Glover Quin
David Bruton
Chip Vaughn
Kyle Moore
Stanley Arnoux
D.J. Moore
Brandon Williams
Shawn Nelson
Anthony Hill
Louis Murphy
Slade Norris
Austin Collie
Tony Fiammetta
Andre Brown
Gerald McRath
Gartrell Johnson
Terrance Taylor
Jason Phillips
Kenny McKinley
DeAngelo Smith
Jarett Dillard
Quinn Johnson
Scott McKillop
Nic Harris
Jasper Brinkley
James Casey
Marcus Freeman
Victor Harris
Cody Glenn
Chris Clemons
Michael Hamlin
Joe Burnett
Frank Summers
Javon Ringer
Tom Brandstater
Spencer Adkins
Morgan Trent
Jarius Wynn
Cary Harris
Bear Pascoe
Cedric Peerman
Robert Henson
Brandon Underwood
Brice McCain
Kevin Ellison
Al Afalava
Coye Francies
Aaron B

Tommy Streeter
Terrance Ganaway
Emmanuel Acho
Billy Winn
LaVon Brazill
Aaron Brown
Audie Cole
Scott Solomon
Michael Smith
Richard Crawford
Tim Fugger
Kheeston Randall
D.J. Campbell
Jerome Long
Trevor Guyton
Greg McCoy
Travis Lewis
Alfonzo Dennard
J.R. Sweezy
Bryce Brown
Toney Clemons
Greg Scruggs
Drake Dunsmore
Jeremy Ebert
DeAngelo Tyson
Cam Johnson
Junior Hemingway
Markus Kuhn
David Paulson
Antonio Allen
Trevin Wade
Terrence Frederick
Brad Smelley
Travian Robertson
Edwin Baker
Chandler Harnish
Dion Jordan
Ezekiel Ansah
Barkevious Mingo
Tavon Austin
Dee Milliner
D.J. Hayden
Sheldon Richardson
Star Lotulelei
Kenny Vaccaro
EJ Manuel
Jarvis Jones
Eric Reid
Tyler Eifert
Desmond Trufant
Sharrif Floyd
Bjoern Werner
Xavier Rhodes
Datone Jones
DeAndre Hopkins
Sylvester Williams
Cordarrelle Patterson
Alec Ogletree
Matt Elam
Johnathan Cyprien
Justin Hunter
Zach Ertz
Darius Slay
Giovani Bernard
Manti Te'o
Geno Smith
Tank Carradine
Robert Woods
Johnthan Banks
Kawann Short
Kevin Minter
Kiko Alonso

Reggie Ragland
Kamalei Correa
Austin Johnson
Jihad Ward
Derrick Henry
A'Shawn Robinson
Michael Thomas
Jarran Reed
Christian Hackenberg
Deion Jones
Su'a Cravens
Mackensie Alexander
Tyler Boyd
T.J. Green
Sean Davis
Cyrus Jones
Vonn Bell
James Bradberry
Adam Gotsis
Kevin Byard
Carl Nassib
Maliek Collins
Will Redmond
Yannick Ngakoue
Bronson Kaufusi
Darian Thompson
Jonathan Bullard
Kenyan Drake
KeiVarae Russell
Shilique Calhoun
Daryl Worley
Adolphus Washington
Austin Hooper
Jordan Jenkins
Kendall Fuller
Braxton Miller
Leonte Carroo
Nick Vigil
Kyler Fackrell
C.J. Prosise
Jacoby Brissett
Brandon Williams
Cody Kessler
Nick Vannett
Vincent Valentine
Justin Simmons
Joe Schobert
Connor Cook
Charles Tapper
Joshua Perry
Sheldon Day
Tavon Young
Eric Murray
Chris Moore
B.J. Goodson
Tyler Higbee
Malcolm Mitchell
Nick Kwiatkoski
Ricardo Louis
De'Vondre Campbell
Hassan Ridgeway
Pharoh Cooper
Juston Burris
Tyler Ervin
Andrew Billings
Deon Bush
Antonio Morrison
Demarcus Robinson
Derrick Kindred
Blake Mart

In [261]:
all_drafts.columns

Index(['college_stats_url', 'draft_age', 'draft_overall', 'draft_round',
       'draft_year', 'player', 'position', 'pro_stats_url', 'school',
       'team_nfl', 'nfl_team', 'nfl_team_win_perc', 'bench_reps', 'broad_jump',
       'cone', 'forty_yd', 'height', 'shuttle', 'vertical', 'weight',
       'games_y1', 'ov_grade_y1', 'games_y2', 'ov_grade_y2', 'games_y3',
       'ov_grade_y3', 'games_y4', 'ov_grade_y4'],
      dtype='object')

In [262]:
all_drafts[['player', 'position', 'draft_year', 'games_y1', 'ov_grade_y1','games_y2', 'ov_grade_y2', 'games_y3', 
            'ov_grade_y3', 'games_y4', 'ov_grade_y4']].head(20)

player position  draft_year  games_y1  ov_grade_y1  games_y2  \
0      Mario Williams       DE        2006      16.0         57.8      16.0   
1         Reggie Bush       RB        2006      16.0         64.5      12.0   
2         Vince Young       QB        2006      15.0         50.0      15.0   
3           A.J. Hawk       LB        2006      16.0         74.1      16.0   
4        Vernon Davis       TE        2006      10.0         69.6      14.0   
5        Michael Huff       DB        2006      16.0         58.1      16.0   
6       Donte Whitner       DB        2006      15.0         66.1      15.0   
7          Ernie Sims       LB        2006      16.0         50.7      16.0   
8        Matt Leinart       QB        2006      12.0         58.4       5.0   
9          Jay Cutler       QB        2006       5.0         51.2      16.0   
10       Haloti Ngata       DT        2006      16.0         69.3      16.0   
11   Kamerion Wimbley       DE        2006      16.0         70.3      16.0   
12   Brodrick Bunkley       DT        2006      15.0         57.1      15.0   
13           Tye Hill       DB        2006      16.0         63.9       8.0   
14        Jason Allen       DB        2006      10.0         60.7      12.0   
15      Chad Greenway       LB        2006       NaN          NaN      16.0   
16    Bobby Carpenter       LB        2006      12.0         56.5       7.0   
17  Antonio Cromartie       DB        2006      16.0         65.7      16.0   
18         Tamba Hali       DE        2006      16.0         71.2      16.0   
19   Laurence Maroney       RB        2006      14.0         55.3      13.0   

    ov_grade_y2  games_y3  ov_grade_y3  games_y4  ov_grade_y4  
0          70.1      16.0         69.4      16.0         69.9  
1          59.5      10.0         54.6      14.0         66.7  
2          68.4       3.0         36.5       3.0         36.5  
3          60.2      16.0         53.8      16.0         65.0  
4          64.9      16.0         65.3      16.0         70.0  
5          52.6      15.0         52.9      16.0         64.7  
6          60.3      13.0         58.2      10.0         57.7  
7          39.4      16.0         30.5      11.0         38.5  
8          50.5       4.0         34.8       4.0         34.8  
9          70.7      16.0         79.3      16.0         79.3  
10         76.2      16.0         69.0      14.0         72.7  
11         69.0      16.0         52.7      15.0         54.1  
12         69.0      16.0         77.6      16.0         74.0  
13         75.7       4.0         33.7       5.0         33.2  
14         61.1       9.0         58.8       4.0         66.4  
15         66.2      16.0         68.2      16.0         68.0  
16         39.5       5.0         70.6      16.0         52.2  
17         89.9      16.0         54.7      16.0         67.3  
18         62.7      15.0         66.2      16.0         66.6  
19         77.7       3.0         56.8      15.0         44.2

In [263]:
all_drafts.isnull().sum()

college_stats_url       0
draft_age              51
draft_overall           0
draft_round             0
draft_year              0
player                  0
position                0
pro_stats_url          47
school                  0
team_nfl                0
nfl_team                0
nfl_team_win_perc       0
bench_reps            417
broad_jump            389
cone                  458
forty_yd              384
height                337
shuttle               453
vertical              388
weight                337
games_y1              594
ov_grade_y1           594
games_y2              744
ov_grade_y2           744
games_y3             1030
ov_grade_y3          1030
games_y4             1263
ov_grade_y4          1263
dtype: int64

In [264]:
len(all_drafts)

2339

I created the below variables to examine how many players in my dataframe did not end up playing any years in the NFL.  I did not drop these however because those would be important datapoints, indicating that these players were not successful compared to someone who did play. 

In [265]:
all_drafts['played_y1'] = all_drafts['games_y1'].map(lambda x: 1 if np.isnan(x) == False else 0)

In [266]:
all_drafts['played_y2'] = all_drafts['games_y2'].map(lambda x: 1 if np.isnan(x) == False else 0)

In [267]:
all_drafts['played_y3'] = all_drafts['games_y3'].map(lambda x: 1 if np.isnan(x) == False else 0)

In [268]:
all_drafts['played_y4'] = all_drafts['games_y4'].map(lambda x: 1 if np.isnan(x) == False else 0)

In [269]:
all_drafts['years_played'] = all_drafts.apply(lambda row: row['played_y1'] + row['played_y2'] + row['played_y3'] + row['played_y4'], axis=1)

In [270]:
all_drafts[['player', 'played_y1', 'played_y2', 'played_y3', 'played_y4', 'years_played']]

player  played_y1  played_y2  played_y3  played_y4  \
0        Mario Williams          1          1          1          1   
1           Reggie Bush          1          1          1          1   
2           Vince Young          1          1          1          1   
3             A.J. Hawk          1          1          1          1   
4          Vernon Davis          1          1          1          1   
5          Michael Huff          1          1          1          1   
6         Donte Whitner          1          1          1          1   
7            Ernie Sims          1          1          1          1   
8          Matt Leinart          1          1          1          1   
9            Jay Cutler          1          1          1          1   
10         Haloti Ngata          1          1          1          1   
11     Kamerion Wimbley          1          1          1          1   
12     Brodrick Bunkley          1          1          1          1   
13             Tye Hill          1          1          1          1   
14          Jason Allen          1          1          1          1   
15        Chad Greenway          0          1          1          1   
16      Bobby Carpenter          1          1          1          1   
17    Antonio Cromartie          1          1          1          1   
18           Tamba Hali          1          1          1          1   
19     Laurence Maroney          1          1          1          1   
20         Manny Lawson          1          1          1          1   
22      Santonio Holmes          1          1          1          1   
23         John McCargo          1          1          1          1   
24    DeAngelo Williams          1          1          1          1   
25       Marcedes Lewis          1          1          1          1   
26         Joseph Addai          1          1          1          1   
27       Kelly Jennings          1          1          1          1   
28     Mathias Kiwanuka          1          1          1          1   
29         DeMeco Ryans          1          1          1          1   
30      D'Qwell Jackson          1          1          1          1   
...                 ...        ...        ...        ...        ...   
2685   Keishawn Bierria          1          0          0          0   
2686          Ade Aruna          0          0          0          0   
2687       Danny Etling          0          0          0          0   
2688       Alex McGough          0          0          0          0   
2689      Matthew Adams          1          0          0          0   
2690     Jermaine Kelly          0          0          0          0   
2691     Jullian Taylor          1          0          0          0   
2692         Javon Wims          1          0          0          0   
2693      Devante Downs          1          0          0          0   
2694     David Williams          1          0          0          0   
2695     Quentin Poling          0          0          0          0   
2696     Marcell Ateman          1          0          0          0   
2697        Leon Jacobs          1          0          0          0   
2698      Travin Howard          0          0          0          0   
2699       James Looney          1          0          0          0   
2700        Andre Smith          1          0          0          0   
2701     Zaire Franklin          1          0          0          0   
2702      Bo Scarbrough          0          0          0          0   
2703        Nick Bawden          0          0          0          0   
2705       Richie James          0          0          0          0   
2706       Greg Stroman          1          0          0          0   
2707    Kendrick Norton          0          0          0          0   
2709      Justin Lawler          1          0          0          0   
2710     Joshua Frazier          0          0          0          0   
2712     Logan Woodside          0       

In [271]:
all_drafts['years_played'].value_counts()

4    857
3    387
2    385
1    366
0    344
Name: years_played, dtype: int64

In [272]:
college_stats = pd.read_csv('../data/college/all_player_stats.csv').drop(columns='Unnamed: 0')

In [273]:
college_stats.head()

player  year                school conference  \
0  Mario Williams  2005  North Carolina State        ACC   
1     Reggie Bush  2003                   USC     Pac-10   
2     Reggie Bush  2004                   USC     Pac-10   
3     Reggie Bush  2005                   USC     Pac-10   
4     Vince Young  2003                 Texas     Big 12   

                                       team_year_url class pos  games  \
0  https://www.sports-reference.com/cfb/schools/n...    JR  DL   12.0   
1  https://www.sports-reference.com/cfb/schools/s...    FR  RB   13.0   
2  https://www.sports-reference.com/cfb/schools/s...    SO  RB   13.0   
3  https://www.sports-reference.com/cfb/schools/s...    JR  RB   13.0   
4  https://www.sports-reference.com/cfb/schools/t...    FR  QB   12.0   

   pass_cmp  pass_att  ...   punt_ret_yds  punt_ret_yds_per_ret  punt_ret_td  \
0       NaN       NaN  ...            7.0                   7.0          0.0   
1       NaN       NaN  ...            4.0                   2.0          0.0   
2       1.0       1.0  ...          376.0                  15.7          2.0   
3       0.0       2.0  ...          179.0                   9.9          1.0   
4      84.0     143.0  ...            NaN                   NaN          NaN   

   kick_ret  kick_ret_yds  kick_ret_yds_per_ret  kick_ret_td  losses  rank  \
0       NaN           NaN                   NaN          NaN     5.0  37.0   
1      18.0         492.0                  27.3          1.0     1.0   4.0   
2      21.0         537.0                  25.6          0.0     0.0   1.0   
3      28.0         493.0                  17.6          0.0     1.0   2.0   
4       NaN           NaN                   NaN          NaN     3.0  13.0   

   wins  
0   7.0  
1  12.0  
2  13.0  
3  12.0  
4  10.0  

[5 rows x 54 columns]

In [274]:
college_stats.isnull().sum()

player                     0
year                       0
school                     0
conference                 1
team_year_url              5
class                     95
pos                        1
games                    217
pass_cmp                7000
pass_att                7000
pass_cmp_pct            7135
pass_yds                7000
pass_yds_per_att        7135
adj_pass_yds_per_att    7135
pass_td                 7000
pass_int                7000
pass_rating             7135
rush_att                5155
rush_yds                5155
rush_yds_per_att        5662
rush_td                 5155
rec                     4761
rec_yds                 4761
rec_yds_per_rec         5114
rec_td                  4761
scrim_att               4163
scrim_yds               4163
scrim_yds_per_att       4566
scrim_td                4163
tackles_solo            1640
tackles_assists         1640
tackles_total           1640
tackles_loss            1640
sacks                   1640
def_int       

In [275]:
college_stats[['pass_cmp', 'pass_att', 'pass_cmp_pct', 'pass_yds',
       'pass_yds_per_att', 'adj_pass_yds_per_att', 'pass_td', 'pass_int',
       'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_per_att', 'rush_td',
       'rec', 'rec_yds', 'rec_yds_per_rec', 'rec_td', 'scrim_att', 'scrim_yds',
       'scrim_yds_per_att', 'scrim_td', 'tackles_solo', 'tackles_assists',
       'tackles_total', 'tackles_loss', 'sacks', 'def_int', 'def_int_yds',
       'def_int_yds_per_int', 'def_int_td', 'pass_defended', 'fumbles_rec',
       'fumbles_rec_yds', 'fumbles_rec_td', 'fumbles_forced', 'punt_ret',
       'punt_ret_yds', 'punt_ret_yds_per_ret', 'punt_ret_td', 'kick_ret',
       'kick_ret_yds', 'kick_ret_yds_per_ret', 'kick_ret_td']] = college_stats[['pass_cmp', 'pass_att', 'pass_cmp_pct', 'pass_yds',
       'pass_yds_per_att', 'adj_pass_yds_per_att', 'pass_td', 'pass_int',
       'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_per_att', 'rush_td',
       'rec', 'rec_yds', 'rec_yds_per_rec', 'rec_td', 'scrim_att', 'scrim_yds',
       'scrim_yds_per_att', 'scrim_td', 'tackles_solo', 'tackles_assists',
       'tackles_total', 'tackles_loss', 'sacks', 'def_int', 'def_int_yds',
       'def_int_yds_per_int', 'def_int_td', 'pass_defended', 'fumbles_rec',
       'fumbles_rec_yds', 'fumbles_rec_td', 'fumbles_forced', 'punt_ret',
       'punt_ret_yds', 'punt_ret_yds_per_ret', 'punt_ret_td', 'kick_ret',
       'kick_ret_yds', 'kick_ret_yds_per_ret', 'kick_ret_td']].fillna(value=0)

I chose to fill null values in my college stats dataframe with 0 values, because the assumption is that if it was not included on a table on pro-football-reference.com there were not stats accumulated in that category that year for that player.  Rather than having a table full of zeros for multiple categories, that website opted to just have those specific categories missing.  Because of this I have to assume those values are zeros for those players, however this could be a limitation of my modeling later on, and other than manually checking and imputing values I don't really have a way of knowing whether or not a stat is actually missing, or just not accumulated that season meaning it would be a zero. 

In [276]:
college_stats.isnull().sum()

player                    0
year                      0
school                    0
conference                1
team_year_url             5
class                    95
pos                       1
games                   217
pass_cmp                  0
pass_att                  0
pass_cmp_pct              0
pass_yds                  0
pass_yds_per_att          0
adj_pass_yds_per_att      0
pass_td                   0
pass_int                  0
pass_rating               0
rush_att                  0
rush_yds                  0
rush_yds_per_att          0
rush_td                   0
rec                       0
rec_yds                   0
rec_yds_per_rec           0
rec_td                    0
scrim_att                 0
scrim_yds                 0
scrim_yds_per_att         0
scrim_td                  0
tackles_solo              0
tackles_assists           0
tackles_total             0
tackles_loss              0
sacks                     0
def_int                   0
def_int_yds         

I create two dataframes from the larger college stats dataframe that I have, those being avg_college_stats and final_year_college_stats, as these would be my way of creating one single datapoint for each player that I could map to my draft_picks, rather than having multiple lines for different years of college stats.  Later on in modeling I opt to use the avg_college_stats over final_year_college_stats. 

In [277]:
avg_college_stats = college_stats.groupby('player', sort=False).mean()

In [278]:
avg_college_stats.head()

year      games    pass_cmp    pass_att  pass_cmp_pct  \
player                                                                    
Mario Williams  2005.0  12.000000    0.000000    0.000000      0.000000   
Reggie Bush     2004.0  13.000000    0.333333    1.000000     33.333333   
Vince Young     2004.0  12.333333  148.000000  239.333333     61.033333   
A.J. Hawk       2003.5  12.750000    0.000000    0.000000      0.000000   
Vernon Davis    2004.0  11.666667    0.000000    0.000000      0.000000   

                   pass_yds  pass_yds_per_att  adj_pass_yds_per_att  \
player                                                                
Mario Williams     0.000000          0.000000              0.000000   
Reggie Bush       17.333333         17.333333             24.000000   
Vince Young     2013.333333          8.266667              7.566667   
A.J. Hawk          0.000000          0.000000              0.000000   
Vernon Davis       0.000000          0.000000              0.000000   

                  pass_td  pass_int    ...      punt_ret_yds  \
player                                 ...                     
Mario Williams   0.000000  0.000000    ...          7.000000   
Reggie Bush      0.333333  0.000000    ...        186.333333   
Vince Young     14.666667  9.333333    ...          0.000000   
A.J. Hawk        0.000000  0.000000    ...          0.000000   
Vernon Davis     0.000000  0.000000    ...          0.000000   

                punt_ret_yds_per_ret  punt_ret_td   kick_ret  kick_ret_yds  \
player                                                                       
Mario Williams                   7.0          0.0   0.000000      0.000000   
Reggie Bush                      9.2          1.0  22.333333    507.333333   
Vince Young                      0.0          0.0   0.000000      0.000000   
A.J. Hawk                        0.0          0.0   0.000000      0.000000   
Vernon Davis                     0.0          0.0   1.000000     22.666667   

                kick_ret_yds_per_ret  kick_ret_td    losses       rank  \
player                                                                   
Mario Williams              0.000000     0.000000  5.000000  37.000000   
Reggie Bush                23.500000     0.333333  0.666667   2.333333   
Vince Young                 0.000000     0.000000  1.333333   6.333333   
A.J. Hawk                   0.000000     0.000000  2.000000  10.250000   
Vernon Davis                7.566667     0.000000  5.000000  47.000000   

                     wins  
player                     
Mario Williams   7.000000  
Reggie Bush     12.333333  
Vince Young     11.333333  
A.J. Hawk       10.750000  
Vernon Davis     6.666667  

[5 rows x 48 columns]

In [279]:
max_year = college_stats.groupby('player', sort=False).max()['year']

In [280]:
college_stats['final_year'] = np.nan
for name, year in max_year.items():
    print(name)
    for index, row in college_stats.iterrows():
        if row['player'] == name:
            college_stats.loc[index, 'final_year'] = year

Mario Williams
Reggie Bush
Vince Young
A.J. Hawk
Vernon Davis
Michael Huff
Donte Whitner
Ernie Sims
Matt Leinart
Jay Cutler
Haloti Ngata
Kamerion Wimbley
Brodrick Bunkley
Tye Hill
Jason Allen
Chad Greenway
Bobby Carpenter
Antonio Cromartie
Tamba Hali
Laurence Maroney
Manny Lawson
Santonio Holmes
John McCargo
DeAngelo Williams
Marcedes Lewis
Joseph Addai
Kelly Jennings
Mathias Kiwanuka
Demeco Ryans
D'Qwell Jackson
Chad Jackson
Thomas Howard
Daniel Bullocks
Roman Harper
Sinorice Moss
Lendale White
Joe Klopfenstein
Cedric Griffin
Kellen Clemens
Greg Jennings
Anthony Fasano
Bernard Pollard
Devin Hester
Richard Marshall
Maurice Drew
Tony Scheffler
Tim Jennings
Darryl Tapp
Tarvaris Jackson
Abdul Hodge
Claude Wroten
Ashton Youboty
Leonard Pope
Dusty Dvoracek
Brian Calhoun
Anthony Schlegel
Jon Alston
Travis Wilson
Jerious Norwood
Clint Ingram
Charlie Whitehurst
Derek Hagan
Anthony Smith
Brandon Williams
Brodie Croyle
David Thomas
James Anderson
Maurice Stovall
Frostee Rucker
Dominique Byrd
Wil

Glen Coffee
Deandre Levy
Kevin Barnes
Roy Miller
Derrick Williams
Brandon Tate
Mike Wallace
Asher Allen
Patrick Turner
Jared Cook
Christopher Owens
Deon Butler
Jerraud Powers
Corvey Irvin
Ryan Mouton
Rashad Johnson
Keenan Lewis
Tyrone McKenzie
Chase Coffman
Juaquin Iglesias
Travis Beckum
Stephen McGee
Donald Washington
Kaluka Maiava
Henry Melton
Mike Thomas
Brian Hartline
Victor Butler
Mike Goodson
Glover Quin
David Bruton
Chip Vaughn
Kyle Moore
Stanley Arnoux
D.J. Moore
Shawn Nelson
Anthony Hill
Louis Murphy
Slade Norris
Austin Collie
Tony Fiammetta
Andre Brown
Gerald McRath
Gartrell Johnson
Terrance Taylor
Jason Phillips
Kenny McKinley
Deangelo Smith
Jarett Dillard
Quinn Johnson
Scott McKillop
Nic Harris
Jasper Brinkley
James Casey
Marcus Freeman
Victor Harris
Cody Glenn
Chris Clemons
Michael Hamlin
Joe Burnett
Frank Summers
Javon Ringer
Tom Brandstater
Spencer Adkins
Morgan Trent
Jarius Wynn
Cary Harris
Bear Pascoe
Cedric Peerman
Robert Henson
Brandon Underwood
Brice McCain
Kevin El

Cyrus Gray
B.J. Cunningham
Ryan Lindley
James Hanna
Josh Bush
Danny Trevathan
Markelle Martin
Dan Herron
Charles Mitchell
Marvin McNutt
Jonte Green
Nate Ebner
Tommy Streeter
Terrance Ganaway
Emmanuel Acho
Billy Winn
Lavon Brazill
Audie Cole
Scott Solomon
Michael Smith
Richard Crawford
Tim Fugger
Kheeston Randall
D.J. Campbell
Jerome Long
Trevor Guyton
Greg McCoy
Travis Lewis
Alfonzo Dennard
Jr Sweezy
Bryce Brown
Toney Clemons
Greg Scruggs
Drake Dunsmore
Jeremy Ebert
Deangelo Tyson
Cam Johnson
Junior Hemingway
Markus Kuhn
David Paulson
Antonio Allen
Trevin Wade
Terrence Frederick
Brad Smelley
Travian Robertson
Edwin Baker
Chandler Harnish
Dion Jordan
Ezekiel Ansah
Barkevious Mingo
Tavon Austin
Demarcus Milliner
D.J. Hayden
Sheldon Richardson
Star Lotulelei
Kenny Vaccaro
E.J. Manuel
Jarvis Jones
Eric Reid
Tyler Eifert
Desmond Trufant
Shariff Floyd
Bjoern Werner
Xavier Rhodes
Datone Jones
Deandre Hopkins
Sylvester Williams
Cordarrelle Patterson
Alec Ogletree
Matt Elam
Jonathan Cyprien
Jus

William Fuller
Josh Doctson
Laquon Treadwell
William Jackson III
Artie Burns
Paxton Lynch
Kenneth Clark
Robert Nkemdiche
Vernon Butler
Emmanuel Ogbah
Kevin Dodd
Jaylon Smith
Hunter Henry
Myles Jack
Xavien Howard
Noah Spence
Sterling Shepard
Reggie Ragland
Kamalei Correa
Austin Johnson
Jihad Ward
Derrick Henry
A'Shawn Robinson
Michael Thomas
Jarran Reed
Christian Hackenberg
Deion Jones
Su'A Cravens
Mackensie Alexander
Tyler Boyd
T.J. Green
Sean Davis
Cyrus Jones
Vonn Bell
James Bradberry
Adam Gotsis
Kevin Byard
Carl Nassib
Maliek Collins
Will Redmond
Yannick Ngakoue
Bronson Kaufusi
Darian Thompson
Jonathan Bullard
Kenyan Drake
Keivarae Russell
Shilique Calhoun
Daryl Worley
Adolphus Washington
Austin Hooper
Jordan Jenkins
Kendall Fuller
Braxton Miller
Leonte Carroo
Nick Vigil
Kyler Fackrell
C.J. Prosise
Jacoby Brissett
Cody Kessler
Nick Vannett
Vincent Valentine
Justin Simmons
Joe Schobert
Connor Cook
Charles Tapper
Joshua Perry
Sheldon Day
Tavon Young
Eric Murray
Chris Moore
B.J. Goodso

In [281]:
college_stats[['player', 'year', 'final_year']].head()

player  year  final_year
0  Mario Williams  2005      2005.0
1     Reggie Bush  2003      2005.0
2     Reggie Bush  2004      2005.0
3     Reggie Bush  2005      2005.0
4     Vince Young  2003      2005.0

In [282]:
final_year_college_stats = college_stats[college_stats['year']==college_stats['final_year']]

In [283]:
final_year_college_stats.head()

player  year                school conference  \
0   Mario Williams  2005  North Carolina State        ACC   
3      Reggie Bush  2005                   USC     Pac-10   
6      Vince Young  2005                 Texas     Big 12   
10       A.J. Hawk  2005            Ohio State    Big Ten   
13    Vernon Davis  2005              Maryland        ACC   

                                        team_year_url class pos  games  \
0   https://www.sports-reference.com/cfb/schools/n...    JR  DL   12.0   
3   https://www.sports-reference.com/cfb/schools/s...    JR  RB   13.0   
6   https://www.sports-reference.com/cfb/schools/t...    JR  QB   13.0   
10  https://www.sports-reference.com/cfb/schools/o...    SR  LB   12.0   
13  https://www.sports-reference.com/cfb/schools/m...    JR  TE   11.0   

    pass_cmp  pass_att     ...      punt_ret_yds_per_ret  punt_ret_td  \
0        0.0       0.0     ...                       7.0          0.0   
3        0.0       2.0     ...                       9.9          1.0   
6      212.0     325.0     ...                       0.0          0.0   
10       0.0       0.0     ...                       0.0          0.0   
13       0.0       0.0     ...                       0.0          0.0   

    kick_ret  kick_ret_yds  kick_ret_yds_per_ret  kick_ret_td  losses  rank  \
0        0.0           0.0                   0.0          0.0     5.0  37.0   
3       28.0         493.0                  17.6          0.0     1.0   2.0   
6        0.0           0.0                   0.0          0.0     0.0   1.0   
10       0.0           0.0                   0.0          0.0     2.0   8.0   
13       3.0          68.0                  22.7          0.0     6.0  65.0   

    wins  final_year  
0    7.0      2005.0  
3   12.0      2005.0  
6   13.0      2005.0  
10  10.0      2005.0  
13   5.0      2005.0  

[5 rows x 55 columns]

In [284]:
len(final_year_college_stats)

2325

In [285]:
len(all_drafts)

2339

In [286]:
all_drafts.columns

Index(['college_stats_url', 'draft_age', 'draft_overall', 'draft_round',
       'draft_year', 'player', 'position', 'pro_stats_url', 'school',
       'team_nfl', 'nfl_team', 'nfl_team_win_perc', 'bench_reps', 'broad_jump',
       'cone', 'forty_yd', 'height', 'shuttle', 'vertical', 'weight',
       'games_y1', 'ov_grade_y1', 'games_y2', 'ov_grade_y2', 'games_y3',
       'ov_grade_y3', 'games_y4', 'ov_grade_y4', 'played_y1', 'played_y2',
       'played_y3', 'played_y4', 'years_played'],
      dtype='object')

In [287]:
final_year_college_stats.columns

Index(['player', 'year', 'school', 'conference', 'team_year_url', 'class',
       'pos', 'games', 'pass_cmp', 'pass_att', 'pass_cmp_pct', 'pass_yds',
       'pass_yds_per_att', 'adj_pass_yds_per_att', 'pass_td', 'pass_int',
       'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_per_att', 'rush_td',
       'rec', 'rec_yds', 'rec_yds_per_rec', 'rec_td', 'scrim_att', 'scrim_yds',
       'scrim_yds_per_att', 'scrim_td', 'tackles_solo', 'tackles_assists',
       'tackles_total', 'tackles_loss', 'sacks', 'def_int', 'def_int_yds',
       'def_int_yds_per_int', 'def_int_td', 'pass_defended', 'fumbles_rec',
       'fumbles_rec_yds', 'fumbles_rec_td', 'fumbles_forced', 'punt_ret',
       'punt_ret_yds', 'punt_ret_yds_per_ret', 'punt_ret_td', 'kick_ret',
       'kick_ret_yds', 'kick_ret_yds_per_ret', 'kick_ret_td', 'losses', 'rank',
       'wins', 'final_year'],
      dtype='object')

In [288]:
final_year_df = pd.merge(all_drafts, final_year_college_stats.drop(columns=['year', 'school', 'team_year_url', 'class', 'pos', 'final_year']), how='left', on='player')

In [289]:
len(final_year_df)

2340

In [290]:
final_year_df.head()

college_stats_url  draft_age  \
0  http://www.sports-reference.com/cfb/players/ma...       21.0   
1  http://www.sports-reference.com/cfb/players/re...       21.0   
2  http://www.sports-reference.com/cfb/players/vi...       23.0   
3  http://www.sports-reference.com/cfb/players/aj...       22.0   
4  http://www.sports-reference.com/cfb/players/ve...       22.0   

   draft_overall  draft_round  draft_year          player position  \
0              1            1        2006  Mario Williams       DE   
1              2            1        2006     Reggie Bush       RB   
2              3            1        2006     Vince Young       QB   
3              5            1        2006       A.J. Hawk       LB   
4              6            1        2006    Vernon Davis       TE   

                                       pro_stats_url              school  \
0  https://www.pro-football-reference.com/players...  North Carolina St.   
1  https://www.pro-football-reference.com/players...                 USC   
2  https://www.pro-football-reference.com/players...               Texas   
3  https://www.pro-football-reference.com/players...            Ohio St.   
4  https://www.pro-football-reference.com/players...            Maryland   

  team_nfl  ...  punt_ret_yds  punt_ret_yds_per_ret  punt_ret_td  kick_ret  \
0      HOU  ...           7.0                   7.0          0.0       0.0   
1      NOR  ...         179.0                   9.9          1.0      28.0   
2      TEN  ...           0.0                   0.0          0.0       0.0   
3      GNB  ...           0.0                   0.0          0.0       0.0   
4      SFO  ...           0.0                   0.0          0.0       3.0   

   kick_ret_yds  kick_ret_yds_per_ret  kick_ret_td  losses  rank  wins  
0           0.0                   0.0          0.0     5.0  37.0   7.0  
1         493.0                  17.6          0.0     1.0   2.0  12.0  
2           0.0                   0.0          0.0     0.0   1.0  13.0  
3           0.0                   0.0          0.0     2.0   8.0  10.0  
4          68.0                  22.7          0.0     6.0  65.0   5.0  

[5 rows x 81 columns]

In [291]:
avg_college_stats.columns

Index(['year', 'games', 'pass_cmp', 'pass_att', 'pass_cmp_pct', 'pass_yds',
       'pass_yds_per_att', 'adj_pass_yds_per_att', 'pass_td', 'pass_int',
       'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_per_att', 'rush_td',
       'rec', 'rec_yds', 'rec_yds_per_rec', 'rec_td', 'scrim_att', 'scrim_yds',
       'scrim_yds_per_att', 'scrim_td', 'tackles_solo', 'tackles_assists',
       'tackles_total', 'tackles_loss', 'sacks', 'def_int', 'def_int_yds',
       'def_int_yds_per_int', 'def_int_td', 'pass_defended', 'fumbles_rec',
       'fumbles_rec_yds', 'fumbles_rec_td', 'fumbles_forced', 'punt_ret',
       'punt_ret_yds', 'punt_ret_yds_per_ret', 'punt_ret_td', 'kick_ret',
       'kick_ret_yds', 'kick_ret_yds_per_ret', 'kick_ret_td', 'losses', 'rank',
       'wins'],
      dtype='object')

In [292]:
avg_college_stats = avg_college_stats.reset_index()

In [293]:
avg_df = pd.merge(all_drafts, avg_college_stats.drop(columns='year'), how='left', on='player')

In [294]:
avg_df = pd.merge(avg_df, final_year_df[['player', 'conference']], how='left', on='player')

In [295]:
len(avg_df)

2370

In [296]:
avg_df.head()

college_stats_url  draft_age  \
0  http://www.sports-reference.com/cfb/players/ma...       21.0   
1  http://www.sports-reference.com/cfb/players/re...       21.0   
2  http://www.sports-reference.com/cfb/players/vi...       23.0   
3  http://www.sports-reference.com/cfb/players/aj...       22.0   
4  http://www.sports-reference.com/cfb/players/ve...       22.0   

   draft_overall  draft_round  draft_year          player position  \
0              1            1        2006  Mario Williams       DE   
1              2            1        2006     Reggie Bush       RB   
2              3            1        2006     Vince Young       QB   
3              5            1        2006       A.J. Hawk       LB   
4              6            1        2006    Vernon Davis       TE   

                                       pro_stats_url              school  \
0  https://www.pro-football-reference.com/players...  North Carolina St.   
1  https://www.pro-football-reference.com/players...                 USC   
2  https://www.pro-football-reference.com/players...               Texas   
3  https://www.pro-football-reference.com/players...            Ohio St.   
4  https://www.pro-football-reference.com/players...            Maryland   

  team_nfl     ...     punt_ret_yds_per_ret  punt_ret_td   kick_ret  \
0      HOU     ...                      7.0          0.0   0.000000   
1      NOR     ...                      9.2          1.0  22.333333   
2      TEN     ...                      0.0          0.0   0.000000   
3      GNB     ...                      0.0          0.0   0.000000   
4      SFO     ...                      0.0          0.0   1.000000   

   kick_ret_yds  kick_ret_yds_per_ret  kick_ret_td    losses       rank  \
0      0.000000              0.000000     0.000000  5.000000  37.000000   
1    507.333333             23.500000     0.333333  0.666667   2.333333   
2      0.000000              0.000000     0.000000  1.333333   6.333333   
3      0.000000              0.000000     0.000000  2.000000  10.250000   
4     22.666667              7.566667     0.000000  5.000000  47.000000   

        wins  conference  
0   7.000000         ACC  
1  12.333333      Pac-10  
2  11.333333      Big 12  
3  10.750000     Big Ten  
4   6.666667         ACC  

[5 rows x 81 columns]

In [297]:
final_year_df.isnull().sum()

college_stats_url          0
draft_age                 51
draft_overall              0
draft_round                0
draft_year                 0
player                     0
position                   0
pro_stats_url             47
school                     0
team_nfl                   0
nfl_team                   0
nfl_team_win_perc          0
bench_reps               417
broad_jump               389
cone                     458
forty_yd                 384
height                   337
shuttle                  453
vertical                 388
weight                   337
games_y1                 594
ov_grade_y1              594
games_y2                 744
ov_grade_y2              744
games_y3                1030
ov_grade_y3             1030
games_y4                1263
ov_grade_y4             1263
played_y1                  0
played_y2                  0
                        ... 
rec_td                   101
scrim_att                101
scrim_yds                101
scrim_yds_per_

In [298]:
final_year_df[final_year_df['conference'].isnull()==True]

college_stats_url  draft_age  \
28    http://www.sports-reference.com/cfb/players/de...       22.0   
35    http://www.sports-reference.com/cfb/players/le...       21.0   
44    http://www.sports-reference.com/cfb/players/ma...       21.0   
112   http://www.sports-reference.com/cfb/players/de...        NaN   
146   http://www.sports-reference.com/cfb/players/la...       22.0   
151   http://www.sports-reference.com/cfb/players/ry...       23.0   
179   http://www.sports-reference.com/cfb/players/la...       22.0   
182   http://www.sports-reference.com/cfb/players/te...       22.0   
214   http://www.sports-reference.com/cfb/players/la...       22.0   
240   http://www.sports-reference.com/cfb/players/mi...       22.0   
311   http://www.sports-reference.com/cfb/players/re...       23.0   
338   http://www.sports-reference.com/cfb/players/de...       23.0   
355   http://www.sports-reference.com/cfb/players/ch...       22.0   
356   http://www.sports-reference.com/cfb/players/jo...       22.0   
395   http://www.sports-reference.com/cfb/players/de...       21.0   
404   http://www.sports-reference.com/cfb/players/pa...       24.0   
422   http://www.sports-reference.com/cfb/players/da...       22.0   
436   http://www.sports-reference.com/cfb/players/th...       23.0   
441   http://www.sports-reference.com/cfb/players/wi...       22.0   
463   http://www.sports-reference.com/cfb/players/ke...       23.0   
469   http://www.sports-reference.com/cfb/players/de...       22.0   
475   http://www.sports-reference.com/cfb/players/ma...       22.0   
501   http://www.sports-reference.com/cfb/players/de...       23.0   
507   http://www.sports-reference.com/cfb/players/mi...       22.0   
532   http://www.sports-reference.com/cfb/players/jo...       22.0   
563   http://www.sports-reference.com/cfb/players/ch...       21.0   
578   http://www.sports-reference.com/cfb/players/mi...       22.0   
597   http://www.sports-reference.com/cfb/players/de...       22.0   
606   http://www.sports-reference.com/cfb/players/ch...       22.0   
644   http://www.sports-reference.com/cfb/players/de...       23.0   
...                                                 ...        ...   
1810  http://www.sports-reference.com/cfb/players/su...       21.0   
1828  http://www.sports-reference.com/cfb/players/ke...       22.0   
1880  http://www.sports-reference.com/cfb/players/de...       23.0   
1884  http://www.sports-reference.com/cfb/players/ma...       22.0   
1948  http://www.sports-reference.com/cfb/players/pr...       23.0   
1953  http://www.sports-reference.com/cfb/players/mi...       23.0   
1969  http://www.sports-reference.com/cfb/players/ad...       21.0   
1977  http://www.sports-reference.com/cfb/players/tr...       22.0   
1996  http://www.sports-reference.com/cfb/players/de...       22.0   
1998  http://www.sports-reference.com/cfb/players/ja...       21.0   
2017  http://www.sports-reference.com/cfb/players/ar...       23.0   
2058  http://www.sports-reference.com/cfb/players/de...       23.0   
2079  http://www.sports-reference.com/cfb/players/na...       23.0   
2090  http://www.sports-reference.com/cfb/players/de...       22.0   
2092  http://www.sports-reference.com/cfb/players/da...       22.0   
2099  http://www.sports-reference.com/cfb/players/mi...       24.0   
2114  http://www.sports-reference.com/cfb/players/jo...       22.0   
2119  http://www.sports-reference.com/cfb/players/le...       22.0   
2124  http://www.sports-reference.com/cfb/players/st...       26.0   
2146  http://www.sports-reference.com/cfb/players/ma...       23.0   
2181  http://www.sports-reference.com/cfb/players/jo...       22.0   
2194  http://www.sports-reference.com/cfb/players/dj...       21.0   
2230  http://www.sports-reference.com/cfb/players/ch...       22.0   
2237  http://www.sports-reference.com/cfb/players/do...       21.0   
2254  http://www.sports-reference.com/cfb/players/rj...       22.0   
2255  http://www.sports-reference.com/cfb/player

In [299]:
avg_df.isnull().sum()

college_stats_url          0
draft_age                 52
draft_overall              0
draft_round                0
draft_year                 0
player                     0
position                   0
pro_stats_url             47
school                     0
team_nfl                   0
nfl_team                   0
nfl_team_win_perc          0
bench_reps               421
broad_jump               393
cone                     463
forty_yd                 389
height                   341
shuttle                  458
vertical                 392
weight                   341
games_y1                 601
ov_grade_y1              601
games_y2                 749
ov_grade_y2              749
games_y3                1044
ov_grade_y3             1044
games_y4                1279
ov_grade_y4             1279
played_y1                  0
played_y2                  0
                        ... 
scrim_att                101
scrim_yds                101
scrim_yds_per_att        101
scrim_td      

#### The remaining null values for my target variables (games_y1, ov_grade_y1... etc.) that come from ProFootballFocus.com are null if there is no data for that player in the given year meaning they did not play a snap that season.  Since I am attempting to model a players immediate effecivness in the NFL, I am hoping to be able to factor out situations where a player did not play rather than give them a grade of zero for that year, considering factors such as getting injured before the season started, or being second string to an established starter or superstar on a given team.  I will however consider players who did not player at all for their first four years in the NFL a failure, as regular rookie contracts are generally 3 or 4 years long.  

Below I am creating one average grade as an average of a players first four years grades in the NFL.  I created logic such that if a player didn't play in a given year ('games_y1 would be null for example) or they played less than 5 games that year, then that year would not be included in the average.

In [300]:
for index, row in final_year_df.iterrows():
    numerator = 0
    denominator = 0
    
    if row['games_y1'] > 4:
        numerator += row['ov_grade_y1']
        denominator += 1
    
    if row['games_y2'] > 4:
        numerator += row['ov_grade_y2']
        denominator += 1
    
    if row['games_y3'] > 4:
        numerator += row['ov_grade_y3']
        denominator += 1
    
    if row['games_y4'] > 4:
        numerator += row['ov_grade_y4']
        denominator += 1

    try:
        final_year_df.loc[index, 'avg_grade'] = numerator / denominator
    except ZeroDivisionError:
        final_year_df.loc[index, 'avg_grade'] = 0

In [301]:
final_year_df.isnull().sum()

college_stats_url          0
draft_age                 51
draft_overall              0
draft_round                0
draft_year                 0
player                     0
position                   0
pro_stats_url             47
school                     0
team_nfl                   0
nfl_team                   0
nfl_team_win_perc          0
bench_reps               417
broad_jump               389
cone                     458
forty_yd                 384
height                   337
shuttle                  453
vertical                 388
weight                   337
games_y1                 594
ov_grade_y1              594
games_y2                 744
ov_grade_y2              744
games_y3                1030
ov_grade_y3             1030
games_y4                1263
ov_grade_y4             1263
played_y1                  0
played_y2                  0
                        ... 
scrim_att                101
scrim_yds                101
scrim_yds_per_att        101
scrim_td      

In [302]:
for index, row in avg_df.iterrows():
    numerator = 0
    denominator = 0
    
    if row['games_y1'] > 4:
        numerator += row['ov_grade_y1']
        denominator += 1
    
    if row['games_y2'] > 4:
        numerator += row['ov_grade_y2']
        denominator += 1
    
    if row['games_y3'] > 4:
        numerator += row['ov_grade_y3']
        denominator += 1
    
    if row['games_y4'] > 4:
        numerator += row['ov_grade_y4']
        denominator += 1

    try:
        avg_df.loc[index, 'avg_grade'] = numerator / denominator
    except ZeroDivisionError:
        avg_df.loc[index, 'avg_grade'] = 0

In [303]:
avg_df.isnull().sum()

college_stats_url          0
draft_age                 52
draft_overall              0
draft_round                0
draft_year                 0
player                     0
position                   0
pro_stats_url             47
school                     0
team_nfl                   0
nfl_team                   0
nfl_team_win_perc          0
bench_reps               421
broad_jump               393
cone                     463
forty_yd                 389
height                   341
shuttle                  458
vertical                 392
weight                   341
games_y1                 601
ov_grade_y1              601
games_y2                 749
ov_grade_y2              749
games_y3                1044
ov_grade_y3             1044
games_y4                1279
ov_grade_y4             1279
played_y1                  0
played_y2                  0
                        ... 
scrim_yds                101
scrim_yds_per_att        101
scrim_td                 101
tackles_solo  

In [313]:
final_year_df.columns

Index(['college_stats_url', 'draft_age', 'draft_overall', 'draft_round',
       'draft_year', 'player', 'position', 'pro_stats_url', 'school',
       'team_nfl', 'nfl_team', 'nfl_team_win_perc', 'bench_reps', 'broad_jump',
       'cone', 'forty_yd', 'height', 'shuttle', 'vertical', 'weight',
       'games_y1', 'ov_grade_y1', 'games_y2', 'ov_grade_y2', 'games_y3',
       'ov_grade_y3', 'games_y4', 'ov_grade_y4', 'played_y1', 'played_y2',
       'played_y3', 'played_y4', 'years_played', 'conference', 'games',
       'pass_cmp', 'pass_att', 'pass_cmp_pct', 'pass_yds', 'pass_yds_per_att',
       'adj_pass_yds_per_att', 'pass_td', 'pass_int', 'pass_rating',
       'rush_att', 'rush_yds', 'rush_yds_per_att', 'rush_td', 'rec', 'rec_yds',
       'rec_yds_per_rec', 'rec_td', 'scrim_att', 'scrim_yds',
       'scrim_yds_per_att', 'scrim_td', 'tackles_solo', 'tackles_assists',
       'tackles_total', 'tackles_loss', 'sacks', 'def_int', 'def_int_yds',
       'def_int_yds_per_int', 'def_int_td', 'pa

In [314]:
final_year_df.drop(columns=['college_stats_url', 'draft_age', 'pro_stats_url', 'team_nfl', 'games_y1', 'ov_grade_y1', 'games_y2', 
                         'ov_grade_y2', 'games_y3', 'ov_grade_y3', 'games_y4', 'ov_grade_y4', 'played_y1', 'played_y2',
                         'played_y3', 'played_y4', 'years_played'], axis=1).columns

Index(['draft_overall', 'draft_round', 'draft_year', 'player', 'position',
       'school', 'nfl_team', 'nfl_team_win_perc', 'bench_reps', 'broad_jump',
       'cone', 'forty_yd', 'height', 'shuttle', 'vertical', 'weight',
       'conference', 'games', 'pass_cmp', 'pass_att', 'pass_cmp_pct',
       'pass_yds', 'pass_yds_per_att', 'adj_pass_yds_per_att', 'pass_td',
       'pass_int', 'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_per_att',
       'rush_td', 'rec', 'rec_yds', 'rec_yds_per_rec', 'rec_td', 'scrim_att',
       'scrim_yds', 'scrim_yds_per_att', 'scrim_td', 'tackles_solo',
       'tackles_assists', 'tackles_total', 'tackles_loss', 'sacks', 'def_int',
       'def_int_yds', 'def_int_yds_per_int', 'def_int_td', 'pass_defended',
       'fumbles_rec', 'fumbles_rec_yds', 'fumbles_rec_td', 'fumbles_forced',
       'punt_ret', 'punt_ret_yds', 'punt_ret_yds_per_ret', 'punt_ret_td',
       'kick_ret', 'kick_ret_yds', 'kick_ret_yds_per_ret', 'kick_ret_td',
       'losses', 'rank', 'wi

In [315]:
final_year_modeling_df = final_year_df[['draft_overall', 'draft_round', 'draft_year', 'player', 'position',
       'school', 'conference', 'wins', 'losses', 'rank', 
       'nfl_team', 'nfl_team_win_perc', 'bench_reps', 'broad_jump',
       'cone', 'forty_yd', 'height', 'shuttle', 'vertical', 'weight',
       'games', 'pass_cmp', 'pass_att', 'pass_cmp_pct',
       'pass_yds', 'pass_yds_per_att', 'adj_pass_yds_per_att', 'pass_td',
       'pass_int', 'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_per_att',
       'rush_td', 'rec', 'rec_yds', 'rec_yds_per_rec', 'rec_td', 'scrim_att',
       'scrim_yds', 'scrim_yds_per_att', 'scrim_td', 'tackles_solo',
       'tackles_assists', 'tackles_total', 'tackles_loss', 'sacks', 'def_int',
       'def_int_yds', 'def_int_yds_per_int', 'def_int_td', 'pass_defended',
       'fumbles_rec', 'fumbles_rec_yds', 'fumbles_rec_td', 'fumbles_forced',
       'punt_ret', 'punt_ret_yds', 'punt_ret_yds_per_ret', 'punt_ret_td',
       'kick_ret', 'kick_ret_yds', 'kick_ret_yds_per_ret', 'kick_ret_td', 'avg_grade']]

In [316]:
final_year_modeling_df.to_csv('../data/final_year_modeling_data.csv')

In [317]:
avg_df.columns

Index(['college_stats_url', 'draft_age', 'draft_overall', 'draft_round',
       'draft_year', 'player', 'position', 'pro_stats_url', 'school',
       'team_nfl', 'nfl_team', 'nfl_team_win_perc', 'bench_reps', 'broad_jump',
       'cone', 'forty_yd', 'height', 'shuttle', 'vertical', 'weight',
       'games_y1', 'ov_grade_y1', 'games_y2', 'ov_grade_y2', 'games_y3',
       'ov_grade_y3', 'games_y4', 'ov_grade_y4', 'played_y1', 'played_y2',
       'played_y3', 'played_y4', 'years_played', 'games', 'pass_cmp',
       'pass_att', 'pass_cmp_pct', 'pass_yds', 'pass_yds_per_att',
       'adj_pass_yds_per_att', 'pass_td', 'pass_int', 'pass_rating',
       'rush_att', 'rush_yds', 'rush_yds_per_att', 'rush_td', 'rec', 'rec_yds',
       'rec_yds_per_rec', 'rec_td', 'scrim_att', 'scrim_yds',
       'scrim_yds_per_att', 'scrim_td', 'tackles_solo', 'tackles_assists',
       'tackles_total', 'tackles_loss', 'sacks', 'def_int', 'def_int_yds',
       'def_int_yds_per_int', 'def_int_td', 'pass_defended', 

In [318]:
avg_df.drop(columns=['college_stats_url', 'draft_age', 'pro_stats_url', 'team_nfl', 'games_y1', 'ov_grade_y1', 'games_y2', 
                         'ov_grade_y2', 'games_y3', 'ov_grade_y3', 'games_y4', 'ov_grade_y4', 'played_y1', 'played_y2',
                         'played_y3', 'played_y4', 'years_played'], axis=1).columns

Index(['draft_overall', 'draft_round', 'draft_year', 'player', 'position',
       'school', 'nfl_team', 'nfl_team_win_perc', 'bench_reps', 'broad_jump',
       'cone', 'forty_yd', 'height', 'shuttle', 'vertical', 'weight', 'games',
       'pass_cmp', 'pass_att', 'pass_cmp_pct', 'pass_yds', 'pass_yds_per_att',
       'adj_pass_yds_per_att', 'pass_td', 'pass_int', 'pass_rating',
       'rush_att', 'rush_yds', 'rush_yds_per_att', 'rush_td', 'rec', 'rec_yds',
       'rec_yds_per_rec', 'rec_td', 'scrim_att', 'scrim_yds',
       'scrim_yds_per_att', 'scrim_td', 'tackles_solo', 'tackles_assists',
       'tackles_total', 'tackles_loss', 'sacks', 'def_int', 'def_int_yds',
       'def_int_yds_per_int', 'def_int_td', 'pass_defended', 'fumbles_rec',
       'fumbles_rec_yds', 'fumbles_rec_td', 'fumbles_forced', 'punt_ret',
       'punt_ret_yds', 'punt_ret_yds_per_ret', 'punt_ret_td', 'kick_ret',
       'kick_ret_yds', 'kick_ret_yds_per_ret', 'kick_ret_td', 'losses', 'rank',
       'wins', 'conferen

In [319]:
avg_modeling_df = avg_df[['draft_overall', 'draft_round', 'draft_year', 'player', 'position',
       'school', 'conference', 'wins', 'losses', 'rank', 
       'nfl_team', 'nfl_team_win_perc', 'bench_reps', 'broad_jump',
       'cone', 'forty_yd', 'height', 'shuttle', 'vertical', 'weight',
       'games', 'pass_cmp', 'pass_att', 'pass_cmp_pct',
       'pass_yds', 'pass_yds_per_att', 'adj_pass_yds_per_att', 'pass_td',
       'pass_int', 'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_per_att',
       'rush_td', 'rec', 'rec_yds', 'rec_yds_per_rec', 'rec_td', 'scrim_att',
       'scrim_yds', 'scrim_yds_per_att', 'scrim_td', 'tackles_solo',
       'tackles_assists', 'tackles_total', 'tackles_loss', 'sacks', 'def_int',
       'def_int_yds', 'def_int_yds_per_int', 'def_int_td', 'pass_defended',
       'fumbles_rec', 'fumbles_rec_yds', 'fumbles_rec_td', 'fumbles_forced',
       'punt_ret', 'punt_ret_yds', 'punt_ret_yds_per_ret', 'punt_ret_td',
       'kick_ret', 'kick_ret_yds', 'kick_ret_yds_per_ret', 'kick_ret_td', 'avg_grade']]

In [320]:
avg_modeling_df.to_csv('../data/avg_modeling_data.csv')

Reordered and dropped unneeded columns above and then saved the dataframe that is read for analysis and modeling. 